In [438]:
import pandas as pd
from pulp import *
import numpy as np

In [502]:
free_agents=pd.read_csv("free_agents.csv")
raptors=pd.read_csv("raptors.csv")


In [ ]:
free_agents.append(raptors)

In [494]:
set(free_agents.columns)-set(raptors.columns)
#For shooting you want to evaluate the shooters not the PF right?

{'AVG SALARY', 'Player', 'Pos', 'TYPE', 'Tm'}

Note we don't have position of the raptors players

In [440]:
BUDGET=109.14 *10**6

In [441]:
#free_agents=free_agents[free_agents['Player'].isin(raptors['name'])==False].reset_index(drop=True)

In [442]:
# Treat like portfolio optimization problem, where risk is turnover.
# Offensive team vs Defensive team
# Convex model using gorubi
# Include injury rate

In [443]:
#2020 salary was known in 2019 (assumption)
#decision vars for every player.

### Turn non-percentage values to percentages

In [444]:
percentages=free_agents.columns[free_agents.columns.str.contains("%")]
res=[free_agents[percentage]/100 if len(free_agents[free_agents[percentage]>1])>1 else free_agents[percentage] for percentage in percentages]

In [445]:
res=pd.DataFrame(res).T
for col in list(res.columns):
    free_agents[col]=res[col]

In [446]:
free_agents['eFG']= (free_agents['FG'] + 0.5 * free_agents['3P'])/ free_agents['FGA']
free_agents['TOV_Factor']=free_agents['TOV'] / (free_agents['FGA']+ 0.44 * free_agents['FTA']+ free_agents['TOV']) 
free_agents = free_agents.replace(np.nan, 0)
#free_agents['Offensive Rebounding']=free_agents['ORB'] / (free_agents['Opp ORB'] + free_agents['DRB'])
#free_agents['Defensive Rebounding']=free_agents['DRB'] / (Opp ORB + DRB)

### Decision Variables 
Note because we do not have opp ORB and DRB I am using ORB% and DRB %

In [490]:
typeplayer= list(set(free_agents['TYPE']))
positions= list(set(free_agents['Pos']))

playersall = LpVariable.dicts(name='Agent', indexs=range(len(free_agents)), cat=LpBinary) 
playertypes= LpVariable.dicts(name="Player",indexs=(range(15),positions),cat=LpBinary) # Ensure that player has 
playersraptors= LpVariable.dicts(name="RaptorsPlayer",indexs=(range(len(raptors))),cat=LpBinary) # Ensure that player has 
positions = LpVariable.dicts(name='Position', indexs=positions, cat=LpBinary) # Ensure all positions are filled


In [491]:
eFG=lpSum(np.array(free_agents['eFG'])*np.array(list(playersall.values())))
TOV_Factor=lpSum(np.array(free_agents['TOV_Factor'])*np.array(list(playersall.values())))
ORB=lpSum(np.array(free_agents['ORB%'])*np.array(list(playersall.values())))
DRB=lpSum(np.array(free_agents['DRB%'])*np.array(list(playersall.values())))

playersbought=15


### Objective
Note because we cannot do non-linear objectives, we always have to have a team size of 15

In [492]:
prob = LpProblem("NBA")
prob.sense = LpMaximize



#divide by playersbought for gurobi instead of 15?
objective=lpSum((.4*eFG)/playersbought - (.25*TOV_Factor)/playersbought + (.2*ORB)/playersbought + (.15*DRB)/playersbought)
prob+=objective

### Constraints

budget constraint: Note that some player's avg salaries are missing. Ie. Zaza has been playing for years but doesn't have an average salary. Replaced with a minimum salary of $899831

In [501]:
raptornonfreeagentsold

{0: Raptors_NonAgent_0,
 2: Raptors_NonAgent_2,
 4: Raptors_NonAgent_4,
 5: Raptors_NonAgent_5,
 6: Raptors_NonAgent_6,
 7: Raptors_NonAgent_7,
 8: Raptors_NonAgent_8,
 11: Raptors_NonAgent_11,
 13: Raptors_NonAgent_13,
 14: Raptors_NonAgent_14}

In [500]:
#budget constraint: 
averagesalary = [int(salary) for salary in free_agents['AVG SALARY'] if salary != "-"]
#Need to fill in NAs
minimumsalaries =898310 #[898310,1445697,1620564,1678854,1737145,1882867,2028594,2174318,2320044,2331593,2564753]
free_agents['AVG SALARY'] = [int(salary) if salary != "-" else minimumsalary for salary in free_agents['AVG SALARY']]

#ensure that we deduct the salary of non free agents on raptors if we choose to kick them
nonfree=raptors[raptors['free agent']==0]
raptornonfreeagentsold=LpVariable.dicts(name='Raptors_NonAgent', indexs=nonfree.index, cat=LpBinary) 
#inverse=LpConstraint(name="RaptorsnonAgent_kicked",e=1-playersall[i],sense=0,rhs=0) #Constraint that if we keep 1, then this variable must be the inverse 0. Ie no matter what we pay their salary
##notefinished

transactioncost=0

#transactioncost=0
#for indexog,indexfa in zip(raptorsfaog.index,raptorsfa.index):
#    transactioncost += playersall[indexfa]*raptorsfaog['salary_2020'][indexog]

spending=lpSum(np.array(free_agents['AVG SALARY'])*np.array(list(playersall.values()))+transactioncost)


budgetconstraint=LpConstraint(name="budget_constraint",e=spending,sense=-1,rhs=BUDGET)
prob.addConstraint(budgetconstraint)


team constraints must be size 15

In [451]:
teamconstraint=LpConstraint(name="team_size",e=lpSum(np.array(list(playersall.values()))),rhs=15,sense=0)
prob.addConstraint(teamconstraint)

position constraints, 5 guards, 5 forwards and 2 centres

In [452]:
def positionalconstraints(df,keyword="C",cname="Center Constraint",constr=2):
    centres=df[df['Pos'].str.contains(keyword)]
    subcentres=set(centres['Pos'])
    #does a forward Centre count as two?
    centerconstraint=0
    for center in centres.index:
        centerconstraint+=playersall[center]
    centreconstraint=LpConstraint(name=cname,e=centerconstraint,sense=1,rhs=constr)
    return(centreconstraint)
prob.addConstraint(positionalconstraints(free_agents))
prob.addConstraint(positionalconstraints(free_agents,keyword="G",cname="Guard Constraint",constr=5))
prob.addConstraint(positionalconstraints(free_agents,keyword="F",cname="Forward Constraint",constr=5))



In [453]:
prob.solve()

1

In [454]:
def showresults(prob):
    # The status of the solution is printed to the screen
    print("="*30,"\nSolution Status:", LpStatus[prob.status])

    # Results
    obj = value(prob.objective)
    print("The objectve is {}".format(round(obj,2)))

    print("Optimal Solution:")
    for v in prob.variables():
        if(v.varValue==1):
            print(v.name, "=", v.varValue)

    print("Constraints")
    for c in list(prob.constraints.values()):
        print(c.name, "=", c.value())
def sensitivity(prob):
    #Print reduced costs
    o = [{'Variables':v.name,
          'Final Value':v.varValue,
          'Reduced Cost':v.dj,
          }
         for v in prob.variables()]
    print(pd.DataFrame(o),'\n')

    #Print shadow price and constraint slack
    o = [{'Constraint':name,
          'Shadow Price':c.pi,
          'RHS':-c.constant,
          'Slack': c.slack}
         for name, c in prob.constraints.items()]
    print(pd.DataFrame(o))
    
showresults(prob)

Solution Status: Optimal
The objectve is 0.28
Optimal Solution:
Agent_105 = 1.0
Agent_160 = 1.0
Agent_162 = 1.0
Agent_168 = 1.0
Agent_177 = 1.0
Agent_19 = 1.0
Agent_193 = 1.0
Agent_215 = 1.0
Agent_219 = 1.0
Agent_28 = 1.0
Agent_59 = 1.0
Agent_80 = 1.0
Agent_81 = 1.0
Agent_88 = 1.0
Agent_96 = 1.0
Constraints
budget_constraint = -59786939.0
team_size = 0.0
Center_Constraint = 4.0
Guard_Constraint = 0.0
Forward_Constraint = 0.0


In [455]:
sensitivity(prob)

     Variables  Final Value  Reduced Cost
0      Agent_0          0.0      0.012466
1      Agent_1          0.0     -0.008333
2     Agent_10          0.0      0.011790
3    Agent_100          0.0      0.012504
4    Agent_101          0.0      0.013044
..         ...          ...           ...
217   Agent_95          0.0      0.014151
218   Agent_96          1.0      0.018764
219   Agent_97          0.0      0.013368
220   Agent_98          0.0      0.013972
221   Agent_99          0.0      0.015626

[222 rows x 3 columns] 

           Constraint  Shadow Price          RHS       Slack
0   budget_constraint          -0.0  109140000.0  59786939.0
1           team_size          -0.0         15.0        -0.0
2   Center_Constraint          -0.0          2.0        -4.0
3    Guard_Constraint          -0.0          5.0        -0.0
4  Forward_Constraint          -0.0          5.0        -0.0


### Chosen Players

In [461]:
chosen=[]
for var in prob.variables():
    if("Agent" in var.name):
        if(var.varValue==1):
            no=int(var.name.split("_")[1])
            chosen.append(free_agents['Player'][no])
chosen
            

['Jordan Loyd',
 'Ekpe Udoh',
 'Ed Davis',
 'Deyonta Davis',
 'DeAndre Jordan',
 'Troy Caupain',
 'Cheick Diallo',
 'Amile Jefferson',
 'Alan Williams',
 'Thomas Bryant',
 'R.J. Hunter',
 'Marcus Derrickson',
 'Malcolm Brogdon',
 'Kyle Korver',
 'Kenneth Faried']

In [467]:
free_agents[free_agents['Player'].isin(chosen)]

,Unnamed: 0,Player,TYPE,AVG SALARY,Pos,Age,Tm,G,MP,PER,...,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,eFG,TOV_Factor
19,19,Troy Caupain,RFA,898310,PG,23,ORL,4,16,30.7,...,12.2,3.1,0.0,0.0,0.0,30.6,144.0,107,0.622449,0.000000
28,28,Thomas Bryant,RFA,898310,C,21,WAS,72,1496,21.0,...,2.9,0.8,2.1,1.9,4.0,24.0,130.0,112,0.647799,0.096486
59,59,R.J. Hunter,UFA,898310,SG,25,BOS,1,26,21.5,...,5.6,1.9,0.0,0.0,5.6,31.5,131.0,111,0.614108,0.000000
80,80,Marcus Derrickson,UFA,898310,PF,22,GSW,11,67,19.4,...,0.7,0.0,0.7,2.1,6.4,32.7,123.0,116,0.638889,0.077537
81,81,Malcolm Brogdon,RFA,21250000,SG,26,MIL,64,1832,17.8,...,5.2,1.2,0.3,2.3,2.6,25.4,121.0,108,0.573684,0.099930
88,88,Kyle Korver,UFA,2564753,SG-PF,37,TOT,70,1334,11.2,...,2.9,0.9,0.4,2.1,3.8,21.7,109.0,111,0.563218,0.101724
96,96,Kenneth Faried,UFA,898310,C,29,TOT,37,728,21.6,...,1.3,1.1,1.5,2.2,5.5,25.7,125.0,109,0.603933,0.096053
105,105,Jordan Loyd,UFA,898310,PG,25,TOR,12,55,16.3,...,5.2,0.0,0.0,1.7,4.4,25.3,130.0,112,0.557325,0.078616
160,160,Ekpe Udoh,UFA,898310,C,31,UTA,51,320,20.9,...,4.2,1.5,4.6,1.9,5.4,17.8,135.0,100,0.694444,0.129428
162,162,Ed Davis,UFA,4886175,C,29,BRK,81,1446,17.3,...,2.0,1.2,1.1,2.1,7.4,15.5,129.0,105,0.616162,0.146525


Some further constraints and formulatons may include if we know the salary before or after? 

1. The salary has uncertainty, we decide on the roster, but the salary could fluctuate between one standard deviation higher with a 50% chance. So the amount paid depends on the asking price of the player. Ie. pay_xi_m = 10000,pay_xi_l = 25000 -> in the budget constraint (.5 * 10000 + .5 * 25000)*xi.
2. Relative Youth: The youth of a player is something we need to consider. If not much data is on them, a TOV of 0 is seen as good but it really isn't

### Salaries

In [456]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(averagesalary)
fig.show()

In [457]:
free_agents

,Unnamed: 0,Player,TYPE,AVG SALARY,Pos,Age,Tm,G,MP,PER,...,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,eFG,TOV_Factor
0,0,Zaza Pachulia,UFA,898310,C,34,DET,68,878,13.7,...,5.1,1.7,1.0,3.2,8.5,15.0,114.0,107,0.444444,0.187354
1,1,Zach Lofton,RFA,898310,SG,26,DET,1,4,-10.0,...,0.0,12.3,0.0,12.3,0.0,0.0,0.0,90,0.000000,0.500000
2,2,Yante Maten,UFA,898310,PF,22,MIA,2,13,5.7,...,0.0,3.8,0.0,0.0,0.0,7.5,70.0,104,0.253333,0.000000
3,3,Wilson Chandler,UFA,2564753,PF-SF,31,TOT,51,1177,8.5,...,3.3,1.0,0.8,1.8,4.9,12.2,107.0,112,0.527273,0.135952
4,4,Willie Cauley-Stein,UFA,2231920,C,25,SAC,81,2213,18.0,...,4.1,2.0,1.1,1.8,4.8,20.3,118.0,108,0.557692,0.091222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,217,Alex Caruso,RFA,2750000,PG,24,LAL,25,531,13.9,...,6.7,2.1,0.8,3.7,4.7,20.1,109.0,110,0.509868,0.173189
218,218,Alec Burks,UFA,2320044,SG,27,TOT,64,1375,12.7,...,4.6,1.4,0.7,2.3,3.2,20.0,106.0,114,0.467456,0.107477
219,219,Alan Williams,UFA,898310,PF,26,BRK,5,26,32.9,...,5.5,1.8,0.0,1.8,3.7,33.0,128.0,98,0.613445,0.062474
220,220,Al-Farouq Aminu,UFA,9720900,PF,28,POR,81,2292,13.2,...,2.2,1.4,0.7,1.5,3.0,16.1,118.0,109,0.512000,0.095982
